In [2]:
## === Import Libraries === ##
import pysam #count variant alleles from BAM
import pandas as pd #data frames
import numpy as np #arrays
import os #interacting with files
from Bio import SeqIO #reading fasta format
import re #regular expressions
import time
import glob
from collections import Counter
from itertools import combinations
from scipy.stats import binom
from scipy import special

### Helper functions

### Function for reads to include in the analysis. 

In [3]:
def check_read(read):
    """
    Helper function to decide what reads should
    be keep when parsing alignment file with `pysam`. 

    Parameters
    ----------
    read : AlignedSegment
        read from alignment file parsed with `pysam`.

    Returns
    -------
    bool
        True/False if read should be included
        
    """
    # Exclude Quality Failures
    if read.is_qcfail:
        return False
    # Exclude Secondary Mappings
    if read.is_secondary:
        return False
    # Exclude Unmapped Reads
    if read.is_unmapped:
        return False
    else:
        return True
    

In [4]:
def get_ref_sequence(refpath):
    """
    Function to read in the reference sequence as a list.
    """
    return [base.upper() for base in list(SeqIO.parse(refpath, "fasta"))[0].seq]


def count_coverage(bampath,
                   contig, 
                   refpath, 
                   callback_function = check_read, 
                   minimum_QUAL = 25, 
                   minimum_COV = 100): 
    """
    Wrapper around `pysam` count coverage that converts the coverage information into a 
    pandas dataframe. 
    """
    
    # Open alignment with pysam
    with pysam.AlignmentFile(bampath, "rb") as bamfile:
        # Get a dataframe of the counts
        count_df = pd.DataFrame.from_dict({base:counts for base, counts in zip("ACGT",
                                                                               bamfile.count_coverage(contig = contig,
                                                                                                      read_callback = callback_function,
                                                                                                      quality_threshold = minimum_QUAL))})
        # Add the depth at each position
        count_df['DP'] = count_df.sum(axis = 1)
        # Add the position 
        count_df['POS'] = count_df.index + 1
        # Add the reference allele
        count_df['REF'] = get_ref_sequence(refpath)
        # Convert counts to frequency 
        count_df.iloc[:,0:4] = count_df.iloc[:,0:4].div(count_df.DP, axis = 0)
        # Handle any NaNs created by dividing by 0 coverage
        count_df = count_df.fillna(0)
        
        return count_df

    
def coverage_filter(snp_df, threshold = 10): 
    """
    Filter rows that don't meet the heuristic 
    threshold of coverage specified by the user. 
    
    default is set to 10X the reciprocal of the 
    allele frequency.
    """
    return snp_df.assign(COV_FILTER = lambda df: df.DP >= (1/df.AF * threshold))


def observation_filter(snp_df, threshold = 2): 
    """
    Filter for the minimum number of reads where a SNP was observed.
    
    The default is 2 reads containing a SNP.
    """
    return snp_df.assign(OBSV_FILTER = lambda df: df.OBSV >= threshold)


def read_position_filter(snp_df, avg_read_length = 71, percentile = 0.9):
    """
    Filter out SNPs that at the extreme start or end of reads. 
    """
    maxavg = avg_read_length*percentile
    minavg = avg_read_length*(1-percentile)
    return snp_df.assign(READPOS_FILTER = lambda df: (df.MEAN_READPOS >= minavg) & (df.MEAN_READPOS <= maxavg))


def strand_bias_filter(snp_df, threshold = 0.9):
    """
    Filter out SNPs with a strand bias
    """
    return snp_df.assign(STRANDBIAS_FILTER = lambda df: (df.STRAND_RATIO >= (1-threshold)) & (df.STRAND_RATIO <= threshold))


def mask(snp_df, mask = [x for x in range(29860, 29904)]):
    """
    Define a mask of position to exclude in the analysis. 
    
    These could be low complexity, close to the start or 
    end of the genome, etc.. 
    """
    return snp_df.loc[~snp_df.POS.isin(mask)]




### Function for making the consensus sequence to call varaints too. 

In [5]:
def make_consensus(bampath, refpath, contig, outpath, callback_function = check_read,  minimum_QUAL = 25, minimum_COV = 100):
    """
    Take the count_df, which is output from `count_coverage`, and make a consensus seq as a string. 
    
    Parameters for consensus calling are the minimum quality score, coverage, and the reads that
    can be included (marked duplicates, quality fails, etc..) defined by `check_read`. 
    
    Returns consensus as string
    """
    
     # String to hold the final consensus seq
    consensus = ""
    
    # Make the count dataframe 
    count_df = count_coverage(bampath = bampath, 
                               refpath = refpath,
                               contig = contig,
                               callback_function = callback_function,  
                               minimum_QUAL = minimum_QUAL, 
                               minimum_COV = minimum_COV)
    
    # Iterate over each row and determine the conesnus sequence and build consensus string
    for index, row in count_df.iterrows(): 
        if row.DP < minimum_COV:
            consensus += row.REF
        else: 
            consensus += max([(row["A"], "A"), (row["C"], "C"), (row["G"], "G"), (row["T"], "T")],key=lambda x:x[0])[1]

    
    # Write out a fasta file for phylogenetic analysis
    with open(outpath, "w") as outfile:
        outfile.write(">" + contig + "\n" + consensus + "\n")

            
    #return "".join(base for pos, base in enumerate(consensus)) 




### Function for variant calling to consensus sequence. 

In [6]:
def call_variants(bampath, refpath, contig, callback_function = check_read,  minimum_QUAL = 25, minimum_COV = 100, maxdepth = 500):
    """
    Read in BAM file and convert to a dataframe containing the frequency of 
    of any bases present at a given position in the reference genome using the
    `pysam` command `count_coverage`. 

    Parameters
    ----------
    filepath : str
        path to the bam file to be parsed
        
    callback_function : function
        function that decides which reads to keep/exclude
        
    ref : str
        name of the contig to count coverage over
        
    ref_path : str
        path to the reference genome as fasta
         
    minimum_qual : int
        minimum QUAL score to count base observation at a position.

    Returns
    -------
    Pandas.DataFrame
       Data Frame containing the bases represented at each positon in the genome
        
    """
    
    # Make the count dataframe 
    count_df = count_coverage(bampath = bampath, 
                               refpath = refpath,
                               contig = contig,
                               callback_function = callback_function,  
                               minimum_QUAL = minimum_QUAL, 
                               minimum_COV = minimum_COV)

    # Melt the data frame to a longer ('tidy') form
    count_df = pd.melt(count_df, 
                       id_vars=['POS', 'DP', 'REF'],
                       value_vars=[base for base in 'ATGC'],
                       value_name='AF',
                       var_name='ALT')
    
    # Remove anything with 0 coverage.
    count_df = count_df[count_df['AF'] > 0]
    # TRUE/FALSE if it's a SNP
    count_df['SNP'] = np.where(count_df['ALT'] != count_df['REF'], True, False)
    # TRUE/FALSE if it's a consensus base.
    count_df['CONS'] = count_df['AF'].map(lambda x: x >= 0.5)
    # Add the number of times a given allele is observed.
    count_df['OBSV'] = count_df.DP * count_df.AF
    # Filter to only get SNPs
    count_df = count_df.loc[count_df['SNP']]

    # Merge the pileup dict and the snp dict. 
    return count_df

    


### Function to make variant matrix for phasing.

In [7]:
def phase_variants(SNPs_df, bampath, contig, maxdepth = 500):
    
    ## ===== Format inputs and get SNP list ===== ##

    SNPs_set = set(pair for pair in zip(SNPs_df.POS, SNPs_df.ALT))                   
    
    ## ===== Move through the BAM file and get haplotypes ===== ##
    
    # Save the haplotypes in a dictionary
    haplotype_dict = {}

    # Get the start and stop
    start = sorted([pos for pos, alt in SNPs_set])[0]
    stop = sorted([pos for pos, alt in SNPs_set])[-1]

    # Open alignment with pysam
    with pysam.AlignmentFile(bampath, "rb") as bamfile:

        # Get the pileup column for a specific region
        for pileupcolumn in bamfile.pileup(contig, max_depth = maxdepth, start = start, stop = stop, stepper = 'nofilter'):

            # Check the if the position has a target SNP (converted to 0-indexed)
            if pileupcolumn.pos in [pos-1 for pos, alt in SNPs_set]:

                # Iterate over every alignment in the pileup column. 
                for pileupread in pileupcolumn.pileups:

                    # Check if the read is valid and can be parsed
                    if check_read(pileupread.alignment) and not pileupread.is_del and not pileupread.is_refskip:

                        # Save the query name
                        qname = pileupread.alignment.query_name
                        
                        # Save the 1-indexed position
                        pos = pileupcolumn.pos + 1

                        # Save the base at that position in the read
                        alt = pileupread.alignment.query_sequence[pileupread.query_position]

                        # Check if this read has the SNP or not
                        if (pos, alt) in SNPs_set:
                            phase = 1 # The read has the SNP

                        else:
                            phase = 0 # The read doesn't have the SNP

                        # Add the readname to the dictionary  
                        if qname in haplotype_dict.keys():
                            haplotype_dict[qname].append((pos, phase, alt)) 
                        else:
                            haplotype_dict[qname] = [(pos, phase, alt)]
                     
    ## ===== Convert the haplotype dictionary to a dataframe filling in missings ===== ##
    
    # Save the haplotypes with missing values (not covered by reads) filled
    completed_haplotype_dict = {}

    # Iterate over the haplotype dictionary created above
    for qname, alleles in haplotype_dict.items():
        
        # Make a dictionary to fill with observed SNPs for each read
        aa_dict = {tup:"-" for tup in SNPs_set}
        
        # Iterate over all observed alleles
        for allele in alleles:
            
            # If it's 1, then an allele was observed
            if allele[1] == 1:
                # Add this observation based on the key
                aa_dict[(allele[0], allele[2])] = 1
            # If it's a wild type allele 
            elif allele[1] == 0:
                # Check every possible wt allele
                for key in aa_dict.keys():
                    # Check by position
                    if key[0] == allele[0]:
                        # If it's the same positon add a 0
                        aa_dict[key] = 0

        # Add this populated dictionary to the haplotype dictionary
        completed_haplotype_dict[qname] = aa_dict

    
    # Convert this dicitonary into a dataframe and take the transpose
    haplotype_df = pd.DataFrame(completed_haplotype_dict).T
    
    # Replace the '-' with 'NaN'
    haplotype_df = haplotype_df.replace('-', np.nan)
    
    # Return the phased SNP df
    return haplotype_df

### Function to test the probability that haplotype exists based on phase counts. 

In [8]:
# def haplotype_probability(haplotype_dict):
    
#    return (haplotype_dict['01'] * haplotype_dict['10']) / (haplotype_dict['00'] * sum(v for v in haplotype_dict.values()))

### Function to calculate p-value that haplotype exists

In [9]:
def linked_pvalue(haplotype_dict, t = 0.001):
    
    o22 = haplotype_dict['11']
    
    n = sum(v for v in haplotype_dict.values())
    p = t
    k = o22 - 1
    
    return 1 - binom.cdf(k,n,p)
    


### Function to calculate p-value for forbidden pairs

In [10]:
def unlinked_pvalue(haplotype_dict, t = 0.001):
    
    o22 = haplotype_dict['11']
    
    n = sum(v for v in haplotype_dict.values())
    p = t
    k = np.arange(0, o22 + 1)

    binomial = binom.pmf(k,n,p)
    
    return sum(binomial)


### Function to determine linkage from pvalues

In [11]:
def determine_linkage(haplotype_dict, phase_df, P_forbidden = 0.05, P_linked = 0.05):
    
    if linked_pvalue(haplotype_dict, t = 0.001) <= (P_linked/special.binom(len(phase_df.columns), 2)): 

        #print(f"Position {snps[0][0]} and Position {snps[1][0]} are linked \n{haplotype_dict}\n")
        return 1

    else:

        if unlinked_pvalue(haplotype_dict, t = 0.01) <= (P_forbidden/special.binom(len(phase_df.columns), 2)): 

            #print(f"Position {snps[0][0]} and Position {snps[1][0]} are NOT linked \n{haplotype_dict}\n")
            return 0

        else: 

            #print(f"Position {snps[0][0]} and Position {snps[1][0]} are UNDERTERMINED \n{haplotype_dict}\n")
            return "NA"



### Input parameters for testing. 

In [14]:
# Paths 
refpath = "../../config/ref/MeVChiTok-SSPE.fa" 
bampath = "../../results/realigned/Frontal_Cortex_2/Frontal_Cortex_2.sorted.bam"
custom_ref = "./Frontal_Cortex_2.fasta"

# Params
callback_function = check_read
contig = "MeVChiTok" 

# Coverage 
minimum_qual = 25
minimum_cov = 100

In [15]:
# Make the consensus sequence 
make_consensus(bampath = bampath,
               refpath = refpath,
               contig = contig,
               outpath = custom_ref,
               callback_function = check_read,
               minimum_QUAL = 25,
               minimum_COV = 100)


In [16]:
# Call varaints 
variant_df = call_variants(bampath = bampath,
                            refpath = custom_ref,
                            contig = contig,
                            minimum_QUAL = minimum_qual, 
                            minimum_COV = minimum_cov,
                            callback_function = check_read, 
                            maxdepth = 10000)

In [17]:
# Filter based on frequency and minimum number of observations. 
variant_df = variant_df[(variant_df['OBSV'] > 20) & (variant_df['AF'] > 0.02)]


In [18]:
# Makes the phaseing matrix 
outdf = phase_variants(variant_df, 
                       bampath,
                       contig,
                       maxdepth = 10000)

In [19]:
outdf.head()

,6764,12391,4178,5071,3869,4182,1622,1356,4760,5312,...,10718,4006,8920,1328,4087,4700,5050,13558,1620,7304
,A,A,C,A,C,A,A,A,T,C,...,G,C,T,C,C,T,C,T,C,C
K00316:387:HHWKYBBXY:4:1126:18639:34741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K00316:387:HHWKYBBXY:4:1219:27438:26810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K00316:387:HHWKYBBXY:4:2126:28726:36499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K00316:387:HHWKYBBXY:4:2219:25307:7504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
K00316:387:HHWKYBBXY:4:1121:22993:44007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# List to hold the rows
row_list = list()
# Matrix containing all read information 
read_matrix = outdf
# Maximum distance within which SNPs can be phased
max_frag_length = 1000
# Minimum Coverage
min_cov = 100


# All non-redundant comparisons between columns 
comparisons = list(combinations(outdf.columns.tolist(), 2))


# Iterate over the comparisons 
for snps in comparisons: 

    # Check that the SNPs are within the allowable distance
    if abs(snps[0][0] - snps[1][0]) <= max_frag_length:
        # SNPs as list
        snps = [snp for snp in snps]
        # Slice the matrix to get these two SNPs, removing NaN
        phase_df = read_matrix[snps].dropna(thresh=len(snps))
        # Check if there are more bridging reads than the minimum required. 
        if len(phase_df.index) >= min_cov:
        
            # Create a list of our conditions
            conditions = [
                (phase_df[snps[0]] == 1) & (phase_df[snps[1]] == 1), #O22
                (phase_df[snps[0]] == 0) & (phase_df[snps[1]] == 0), #O11
                (phase_df[snps[0]] == 1) & (phase_df[snps[1]] == 0), #O21
                (phase_df[snps[0]] == 0) & (phase_df[snps[1]] == 1)  #O12
                ]

            # Create a list of the values we want to assign for each condition
            values = [('11'), ('00'), ('10'), ('01')]

            # Create a new column and use np.select to assign values to it using our lists as arguments
            phase_df[('haplotype')] = np.select(conditions, values)
            
            # Make a dictionary of O22, O11, O21, and O12
            haplotype_dict = dict(Counter(phase_df.haplotype))
            
            # Add in the missing values
            for k in values: 
                if k not in haplotype_dict.keys():
                    haplotype_dict[k] = 0
                    
            
            if haplotype_dict['11'] >= 10: 
                linkage = determine_linkage(haplotype_dict, outdf, P_forbidden = 0.05, P_linked = 0.001)
            else:
                linkage = "NA"
                
            
            if linkage == 0:
                print(haplotype_dict)
            ## ==== ## ==== ## ==== ##
            
            POS_a = snps[0][0]
            POS_b = snps[1][0]
            
            ALT_a = snps[0][1]
            ALT_b = snps[1][1]
            
            reference_genome = get_ref_sequence(refpath)
            consensus_genome = get_ref_sequence(custom_ref)
            
            REF_a = reference_genome[POS_a - 1]
            REF_b = reference_genome[POS_b - 1]
            
            if REF_a == ALT_a:
                
                ALT_a = consensus_genome[POS_a - 1]
                
                if linkage == 1: 
                    linkage = 0
  
                elif linkage == 0: 
                    linkage = 1

            
            elif REF_b == ALT_b:
                
                ALT_b = consensus_genome[POS_b - 1]
                
                if linkage == 1: 
                    linkage = 0
  
                elif linkage == 0: 
                    linkage = 1
            
            row = [f"{REF_a}{POS_a}{ALT_a}", f"{REF_b}{POS_b}{ALT_b}", linkage, haplotype_dict['11'], haplotype_dict['10'], haplotype_dict['01'], haplotype_dict['00']]
            row_list.append(row)

# Create the pandas DataFrame
df = pd.DataFrame(row_list, columns = ["SNP_a", "SNP_b", "linkage", "O22", "O21" , "O12", "O11"])
 
# print dataframe.
print(df)
df.to_csv("../../config/linkage.csv", index=False)     

{'00': 3603, '10': 3618, '01': 435, '11': 16}
{'00': 5489, '10': 1028, '01': 203, '11': 12}
{'00': 2576, '01': 2197, '10': 890, '11': 14}
{'00': 2493, '01': 2171, '10': 874, '11': 15}
{'00': 5861, '01': 1212, '10': 146, '11': 11}
{'00': 7282, '01': 1363, '10': 288, '11': 16}
{'00': 3775, '01': 881, '10': 161, '11': 16}
{'00': 5358, '01': 1044, '10': 232, '11': 12}
{'00': 7463, '10': 210, '01': 225, '11': 26}
{'00': 2355, '01': 2402, '10': 125, '11': 12}
{'00': 5976, '01': 888, '10': 198, '11': 10}
{'00': 6703, '10': 138, '01': 133, '11': 14}
{'00': 5711, '10': 1197, '01': 335, '11': 22}
{'00': 5337, '10': 1077, '01': 734, '11': 23}
{'10': 2693, '00': 3855, '01': 225, '11': 23}
{'10': 3194, '00': 2879, '01': 1062, '11': 17}
{'00': 4374, '01': 467, '10': 124, '11': 14}
{'00': 4920, '01': 582, '10': 150, '11': 18}
{'00': 4586, '01': 509, '10': 134, '11': 16}
{'00': 3285, '10': 4271, '01': 912, '11': 11}
{'00': 3049, '10': 3928, '01': 862, '11': 14}
{'00': 5933, '10': 979, '01': 387, '11':

KeyboardInterrupt: 

In [192]:

unlinked_pvalue({'00': 501, '01': 13, '10': 15, '11': 2}, t = 0.02)

0.0015448542799962176

In [ ]:
# Path to labeled haplotypes
genomes_path = "../../config/data/allmutations.csv" # Path to the major haplotypes

# Import the dataframe with mutations labled by identity
genomes_df = pd.read_csv(genomes_path)

# Get the SNPs known to belong to genome-1 or genome-2
g1_snps = set(genomes_df[genomes_df.Genome == "genome-1"].SNP)
g2_snps = set(genomes_df[genomes_df.Genome == "genome-2"].SNP)

# SNPs added to the list
diff = abs(len(g1_snps.union(g2_snps)))
# Iteration
iteration = 1

# Keep going until no new SNPs are added 
while diff != 0:
    
    # Lists to hold the SNPs added to either haplotype
    new_g1_snps = list()
    new_g2_snps = list()
    
    print(iteration)
    iteration += 1 
    
    for i in range(len(df)):

        SNP_a = df.iloc[i].SNP_a
        SNP_b = df.iloc[i].SNP_b
        linkage = df.iloc[i].linkage

        if SNP_a in g1_snps and SNP_b not in g1_snps.union(g2_snps): 
            if linkage == 1: 
                new_g1_snps.append(SNP_b)
            elif linkage == 0: 
                new_g2_snps.append(SNP_b)
        elif SNP_a in g2_snps and SNP_b not in g1_snps.union(g2_snps): 
            if linkage == 1: 
                new_g2_snps.append(SNP_b)
            elif linkage == 0: 
                new_g1_snps.append(SNP_b)
        elif SNP_b in g1_snps and SNP_a not in g1_snps.union(g2_snps): 
            if linkage == 1: 
                new_g1_snps.append(SNP_a)
            elif linkage == 0: 
                new_g2_snps.append(SNP_a)
        elif SNP_b in g2_snps and SNP_a not in g1_snps.union(g2_snps): 
            if linkage == 1: 
                new_g2_snps.append(SNP_a)
            elif linkage == 0: 
                new_g1_snps.append(SNP_a)
                
    new_g1_snps = set(new_g1_snps)
    new_g2_snps = set(new_g2_snps)

    diff = abs(len(g1_snps.union(g2_snps)) - len((g1_snps.union(g2_snps)).union((new_g1_snps.union(new_g2_snps)))))
    print(f"Added {diff} new SNPS.")

    g1_snps = g1_snps.union(new_g1_snps)
    g2_snps = g2_snps.union(new_g2_snps)


In [304]:
pd.DataFrame([[snp, "genome-1"] for snp in g1_snps] + [[snp, "genome-2"] for snp in g2_snps],  columns = ["SNP", "Genome"]).to_csv("../../config/gneome_assignments.csv", index = False)

### Use the linkage information to make an adjacency matrix. 

In [26]:
testdf = df[df.linkage == 1]
testdf

,SNP_a,SNP_b,linkage
4,T4345C,T4288C,1
5,T4345C,T4485C,1
11,T4345C,T4280C,1
17,T4345C,T4379C,1
18,T4345C,T4210C,1
...,...,...,...
578,T3670C,T3680C,1
581,T3506C,T3513C,1
583,T3513C,T3456C,1
587,A8004G,A7824G,1


In [66]:
x = testdf.pivot(index='SNP_a', columns='SNP_b')['linkage']

x.fillna(0, inplace=True)
result = x.combine_first(x.T).fillna(0.0)

adj_matrix = [l[1:] for l in result.reset_index().values.tolist()]
snps = result.index.to_list()

In [70]:
N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(adj_matrix)
}


def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        yield R
    while P:
        v = P.pop()
        yield from BronKerbosch1(
            P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v]))
        X.add(v)

P = N.keys()

cliques = [{snps[x] for x in sets} for sets in list(BronKerbosch1(P))]

In [92]:
c_i = {'T6994C', 'T6936C', 'T6945C', 'T7073C'}
c_j = {'T6994C', 'T6945C', 'T7073C'}

for i in c_i:
    for j in c_j:
        print(result[i][j])

1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0


In [72]:
import networkx as nx

In [73]:
G = nx.from_pandas_edgelist(testdf, source="SNP_a", target="SNP_b")

In [75]:
from pyvis.network import Network

In [77]:
net = Network(notebook=True)
net.from_nx(G)
net.show("example.html")